In [ ]:
from mp_api.client import MPRester
import pandas as pd
import numpy as np
import os
from matminer.utils.io import load_dataframe_from_json
from matminer.featurizers.structure import (BondFractions,
                                            MaximumPackingEfficiency,
                                            SiteStatsFingerprint,
                                            DensityFeatures)
from matminer.featurizers.site import VoronoiFingerprint
from matminer.featurizers.composition import (ElementProperty,
                                             Stoichiometry,
                                             IonProperty)
from pymatgen.core.composition import Composition
from pymatgen.core.structure import Structure

# Import query dataset.json
file_path = '/Users/navin/Library/CloudStorage/Dropbox-AIZOTH/研究/Navin/NIMS/surrogate-DFT-ionic-conductivity/src/20250317_model1_improved'
df = load_dataframe_from_json(os.path.join(file_path, 'query_data.json'))

Decoding objects from /Users/navin/Library/CloudStorage/Dropbox-AIZOTH/研究/Navin/NIMS/surrogate-DFT-ionic-conductivity/src/20250317_model1_improved/query_data.json: 0it [00:00, ?it/s]

JSONDecodeError: Extra data: line 2 column 1 (char 5557)

In [19]:

# Convert formula to composition
df['composition'] = df['formula_pretty'].apply(lambda x: Composition(x))

# Function to add oxidation states
# def add_oxidation_states(structure):
    #try:
        #return structure.add_oxidation_state_by_guess()
    #except Exception:
        #return structure  # If it fails, return original structure

#df['structure'] = df['structure'].apply(add_oxidation_states)

/var/folders/72/gw5k58ts05d41x_r9qhs9g680000gn/T/ipykernel_10705/1710305695.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['composition'] = df['formula_pretty'].apply(lambda x: Composition(x))


In [20]:
# Featurizer functions

def apply_bond_fractions(df):
    bf = BondFractions(no_oxi=True)
    bf.fit(df['structure'].dropna().tolist())
    df_expanded = df['structure'].apply(lambda x: bf.featurize(x) if x else None).apply(pd.Series)
    return pd.concat([df, df_expanded], axis=1)

def apply_max_packing_efficiency(df):
    mpe = MaximumPackingEfficiency()
    df_expanded = df['structure'].apply(lambda x: mpe.featurize(x) if x else None).apply(pd.Series)
    return pd.concat([df, df_expanded], axis=1)

def apply_element_properties(df):
    ep = ElementProperty.from_preset("magpie", impute_nan=True)
    df_expanded = df['composition'].apply(lambda x: ep.featurize(x) if x else None).apply(pd.Series)
    return pd.concat([df, df_expanded], axis=1)

def apply_stoichiometry(df):
    stoich = Stoichiometry()
    df_expanded = df['composition'].apply(lambda x: stoich.featurize(x) if x else None).apply(pd.Series)
    return pd.concat([df, df_expanded], axis=1)

def apply_ion_properties(df):
    ionp = IonProperty(impute_nan=True)
    df_expanded = df['composition'].apply(lambda x: ionp.featurize(x) if x else None).apply(pd.Series)
    return pd.concat([df, df_expanded], axis=1)

def apply_density_features(df):
    df_expanded = df['structure'].apply(lambda x: DensityFeatures().featurize(x) if x else None).apply(pd.Series)
    return pd.concat([df, df_expanded], axis=1)

def apply_site_stats_fingerprint(df):
    df_expanded = df['structure'].apply(lambda x: SiteStatsFingerprint(VoronoiFingerprint()).featurize(x) if x else None).apply(pd.Series)
    return pd.concat([df, df_expanded], axis=1)

In [13]:
# Running each feature separately
df = apply_bond_fractions(df)

/opt/anaconda3/lib/python3.11/site-packages/pymatgen/core/periodic_table.py:1061: UserWarning: No Pauling electronegativity for He. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
  return getattr(self._el, attr)
/opt/anaconda3/lib/python3.11/site-packages/pymatgen/core/periodic_table.py:1061: UserWarning: No Pauling electronegativity for Ar. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
  return getattr(self._el, attr)
/opt/anaconda3/lib/python3.11/site-packages/pymatgen/core/periodic_table.py:1061: UserWarning: No Pauling electronegativity for Ne. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
  return getattr(self._el, attr)


ValueError: No Voronoi neighbors found for site - try increasing cutoff

In [21]:
df = apply_max_packing_efficiency(df)

KeyboardInterrupt: 

In [22]:
df = apply_element_properties(df)

In [23]:
df = apply_stoichiometry(df)

/opt/anaconda3/lib/python3.11/site-packages/pymatgen/core/composition.py:1337: UserWarning: No Pauling electronegativity for He. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
  syms = sorted(sym_amt, key=lambda x: [get_el_sp(x).X, x])
/opt/anaconda3/lib/python3.11/site-packages/pymatgen/core/composition.py:1337: UserWarning: No Pauling electronegativity for Ar. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
  syms = sorted(sym_amt, key=lambda x: [get_el_sp(x).X, x])
/opt/anaconda3/lib/python3.11/site-packages/pymatgen/core/composition.py:1337: UserWarning: No Pauling electronegativity for Ne. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
  syms = sorted(sym_amt, key=lambda x: [get_el_sp(x).X, x])


In [25]:
%%time
df = apply_ion_properties(df)

KeyboardInterrupt: 

In [26]:
df = apply_density_features(df)

TypeError: unsupported operand type(s) for ** or pow(): 'NoneType' and 'int'

In [27]:
df = apply_site_stats_fingerprint(df)

ValueError: No Voronoi neighbors found for site - try increasing cutoff